In [1]:
import pandas as pd

import torch
from torch.utils.data import Dataset, DataLoader

import seaborn

import optuna

from sklearn.metrics import r2_score

from common import *

#### Load data

In [2]:
train_data = pd.read_csv("../../data/ld50/train.csv")[["ld50"]]
test_data = pd.read_csv("../../data/ld50/test.csv")[["ld50"]]

In [ ]:
class SmilesDataset(torch.utils.data.Dataset):
    def __init__(self, data: pd.DataFrame, x_label: str, y_label: str):
        self.X = data[x_label]
        self.Y = data[y_label]

    def __len__(self):
        return len(self.Y)
    
    def __getitem__(self, index: int):
        return self.X.iloc[index], self.Y.iloc[index]

train_dataset = SmilesDataset(train_data, "smiles", "ld50")
test_dataset = SmilesDataset(test_data, "smiles", "ld50")
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)

#### Load model and extend layers

In [ ]:
MoLFormer, tokenizer = load_model()
model = torch.nn.Sequential(
    MoLFormer,
    torch.nn.Linear(768, 1024),
    torch.nn.LeakyReLU(),
    torch.nn.Linear(1024, 1024),
    torch.nn.LeakyReLU(),
    torch.nn.Linear(1024, 1)
)